In [23]:
import pandas as pd

df = pd.read_csv("QA.csv", encoding="ISO-8859-1")
df.to_csv("QA_fixed.csv", index=False, encoding="utf-8")


In [24]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="QA_fixed.csv",source_column='prompt')
data = loader.load()

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [26]:
from langchain.vectorstores import FAISS

vecter_db = FAISS.from_documents(documents=data,embedding=embedding)
retriever = vecter_db.as_retriever(score_threshold = 0.7)

In [27]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq.chat_models import ChatGroq
from dotenv import load_dotenv
import os 

In [28]:
load_dotenv()

True

In [29]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [30]:
llm = ChatGroq(api_key=GROQ_API_KEY,temperature=0.7)

In [31]:
system_prompt  = """
Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT
{context}

QUESTION
{question}
"""
PROMPT = PromptTemplate(
    template=system_prompt,input_variables=["context","question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [32]:
chain =RetrievalQA.from_chain_type(llm=llm,
                            chain_type='stuff',
                            retriever =retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [33]:
response = chain('Do you provide job assistance and also do you provide job gurantee?')
response["result"]

'Yes, we do provide job assistance. We help you with resume and interview preparation, building online credibility, and referring candidates to potential recruiters. However, I must clarify that we do not guarantee a job. Our goal is to prepare you for the job market by teaching relevant skills, knowledge, and timeless principles that will increase your chances of securing a job.'

In [34]:
response = chain('do you have javascript course?')
response["result"]

"I don't know. The context provided does not mention any JavaScript course."

In [35]:
response = chain('Do you have plans to launch blockchain course in future?')
response["result"]

"I don't know. The context provided doesn't mention any plans to launch a blockchain course in the future."